In [9]:
import os, sys, time
working_dir = 'H:/UGA MASTERS/SDG/DOL'
os.chdir(working_dir)
import pandas as pd

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.request

#############################################################################
# Extract / Transform
#############################################################################
def excel2csv(outfiles):
    for file in outfiles:
        df = pd.read_excel(file, header=0) #, sheetname='<your sheet>'
        df.to_csv(file.replace('.xlsx','.csv'))        

def get_DOL_data():
    url = 'https://www.dol.gov/agencies/eta/foreign-labor/performance'
    page = urlopen (url)
    soup = BeautifulSoup(page.read())
    links = soup.findAll('a', href=True)
    outfiles = []
    print('Downloading data')
    for link in links:
        if 'LCA_Disclosure' in link['href']: 
            print(f'https://www.dol.gov{link["href"]}')
            href = f'https://www.dol.gov{link["href"]}'
            outname = link["href"].split('/')[-1]
            urllib.request.urlretrieve(href, f'data/{outname}')
            outfiles.append(f'data/{outname}')
    print('Converting to csv')
    excel2csv(outfiles)
    return "Data downloaded and written to .csv"
     
                             
#############################################################################
# Load
#############################################################################            
def SqlLoad():

    try:
        dbconnect = MySQLdb.connect(
                host='localhost',
                user='root',
                passwd='MAX!553MAX',
                db='DOLdb')
    except:
        print('No connection')

    cursor = dbconnect.cursor()
    files = os.listdir("data")    
    files = list(filter(lambda f: f.endswith('.csv'), files))
    for file in files:
        with open(f'data/{file}') as csv_file:

            csv_data = csv.reader(csv_file)
            for row in csv_data:
                cursor.execute(
                'INSERT INTO testDB(CASE_NUMBER, PW_UNIT_OF_PAY, WAGE_RATE_OF_PAY_FROM, JOB_TITLE, \
                        PAY, EMPLOYER_STATE, EMPLOYER_CITY) ' \
                        'VALUES("%s", "%s", "%s", "%s", "%s", "%s",)',
                        row
                        )

        dbconnect.commit()
    cursor.close()
    return 'Read .csv and written to the MySQL database'                 

In [10]:
get_DOL_data()

https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2021_Q2.xlsx
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q1.xlsx
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q2.xlsx
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q3.xlsx
https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q4.xlsx
Converting to csv


'Data downloaded and written to .csv'